In [1]:
import numpy as np
import torch 
import pyro
import re
import pandas as pd

import importlib
import time
import pickle

from granch_utils import init_model_tensor, main_sim_tensor, lesioned_sim, compute_prob_tensor, init_params_tensor, num_stab_help, proxy_sim
#importlib.reload(granch_utils)
importlib.reload(num_stab_help)
importlib.reload(init_model_tensor)
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)

<module 'granch_utils.lesioned_sim' from '/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/lesioned_sim.py'>

In [5]:
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)
importlib.reload(init_params_tensor)
importlib.reload(compute_prob_tensor)
importlib.reload(proxy_sim)


device = 'cuda' if torch.cuda.is_available() else 'cpu'

BATCH_INFO = {
    "jitter_n": 1, 
    "total_batch_n": 1, 
    "jitter_mode": "sampling"
}

GRID_INFO = {
    "grid_mu_start": -4, "grid_mu_end": 4, "grid_mu_step": 20, 
    "grid_sigma_start": 0.001, "grid_sigma_end": 1.8, "grid_sigma_step": 20, 
    "grid_y_start": -4, "grid_y_end": 4, "grid_y_step": 20, 
    "grid_epsilon_start": 0.000001, "grid_epsilon_end": 1, "grid_epsilon_step": 20, 
    "hypothetical_obs_grid_n": 10
}


BATCH_GRID_INFO = num_stab_help.get_batch_grid(BATCH_INFO, GRID_INFO)

PRIOR_INFO = {
    "mu_prior": 1,  
    "V_prior": 1, 
    "alpha_prior": 3, 
    "beta_prior": 1, 
    "epsilon": 0.0001, "mu_epsilon":0.0001, "sd_epsilon": 0.0001, 
    "hypothetical_obs_grid_n": 2, 
    "world_EIGs": 0.0001, "max_observation": 500
}

tensor_stimuli = num_stab_help.sample_spore_experiment(pair_each_stim = 1, n_feature=3)
print(tensor_stimuli)


tensor_model =  init_model_tensor.granch_model(PRIOR_INFO['max_observation'], tensor_stimuli[0])

params = init_params_tensor.granch_params(
                grid_mu =  BATCH_GRID_INFO["grid_mus"][0].to(device),
                grid_sigma = BATCH_GRID_INFO["grid_sigmas"][0].to(device),
                grid_y = BATCH_GRID_INFO["grid_ys"][0].to(device),
                grid_epsilon = BATCH_GRID_INFO["grid_epsilons"][0].to(device),
                hypothetical_obs_grid_n = PRIOR_INFO["hypothetical_obs_grid_n"], 
                mu_prior = PRIOR_INFO["mu_prior"],
                V_prior = PRIOR_INFO["V_prior"], 
                alpha_prior = PRIOR_INFO["alpha_prior"], 
                beta_prior = PRIOR_INFO["beta_prior"],
                epsilon  = PRIOR_INFO["epsilon"], 
                mu_epsilon = PRIOR_INFO["mu_epsilon"], 
                sd_epsilon = PRIOR_INFO["sd_epsilon"], 
                world_EIGs = PRIOR_INFO["world_EIGs"],
                max_observation = PRIOR_INFO["max_observation"], 
                forced_exposure_max= np.nan, 
                linking_hypothesis = "surprisal")
        
            # add the various different cached bits
params.add_meshed_grid()
params.add_lp_mu_sigma()
params.add_y_given_mu_sigma()
params.add_lp_epsilon()
params.add_priors()

if params.linking_hypothesis == "EIG": 
    res = main_sim_tensor.granch_main_simulation(params, tensor_model, tensor_stimuli[0])
else: 
    res = proxy_sim.granch_proxy_sim(params, tensor_model, tensor_stimuli[0])
#res = lesioned_sim.granch_no_learning_simulation(params, tensor_model, tensor_stimuli[0])
#res = lesioned_sim.granch_no_noise_simulation(params, tensor_model, tensor_stimuli[0])



pd.set_option('display.max_rows', None)

print(res.behavior)
#main_sim_tensor.granch_main_simulation(PRIOR_INFO, tensor_model, tensor_stimuli)



[<granch_utils.init_model_tensor.granch_stimuli object at 0x16e2be110>, <granch_utils.init_model_tensor.granch_stimuli object at 0x16e2bfd00>, <granch_utils.init_model_tensor.granch_stimuli object at 0x16e1966b0>, <granch_utils.init_model_tensor.granch_stimuli object at 0x16e195c00>, <granch_utils.init_model_tensor.granch_stimuli object at 0x16e195150>, <granch_utils.init_model_tensor.granch_stimuli object at 0x16e196830>, <granch_utils.init_model_tensor.granch_stimuli object at 0x16e197b80>, <granch_utils.init_model_tensor.granch_stimuli object at 0x16e194730>]
prior
tensor([[[[-43123076.0000,  -8419464.0000, -43321856.0000,  ...,
            -4889013.5000, -10906327.0000, -37112448.0000],
          [-43123076.0000,  -8419464.0000, -43321856.0000,  ...,
            -4889013.5000, -10906327.0000, -37112448.0000],
          [-43123084.0000,  -8419470.0000, -43321864.0000,  ...,
            -4889019.5000, -10906333.0000, -37112456.0000],
          ...,
          [-43123076.0000,  -841946

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:292: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.1969880938620735' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.all_observations.loc[self.current_t]  = Normal(current_stimulus, noise_epsilon).sample().tolist()
/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:292: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.13887756833593223' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.all_observations.loc[self.current_t]  = Normal(current_stimulus, noise_epsilon).sample().tolist()
/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:292: FutureWar

posterior
tensor([[[[9.9999e-321, 9.9999e-321, 9.9999e-321,  ..., 9.9999e-321,
           9.9999e-321, 9.9999e-321],
          [9.9999e-321, 9.9999e-321, 9.9999e-321,  ..., 9.9999e-321,
           9.9999e-321, 9.9999e-321],
          [9.9999e-321, 9.9999e-321, 9.9999e-321,  ..., 9.9999e-321,
           9.9999e-321, 9.9999e-321],
          ...,
          [9.9999e-321, 9.9999e-321, 9.9999e-321,  ..., 9.9999e-321,
           9.9999e-321, 9.9999e-321],
          [9.9999e-321, 9.9999e-321, 9.9999e-321,  ..., 9.9999e-321,
           9.9999e-321, 9.9999e-321],
          [9.9999e-321, 9.9999e-321, 9.9999e-321,  ..., 9.9999e-321,
           9.9999e-321, 9.9999e-321]],

         [[9.9999e-321, 9.9999e-321, 9.9999e-321,  ..., 9.9999e-321,
           9.9999e-321, 9.9999e-321],
          [9.9999e-321, 9.9999e-321, 9.9999e-321,  ..., 9.9999e-321,
           9.9999e-321, 9.9999e-321],
          [9.9999e-321, 9.9999e-321, 9.9999e-321,  ..., 9.9999e-321,
           9.9999e-321, 9.9999e-321],
          

# Test 

In [15]:
importlib.reload(init_model_tensor)
importlib.reload(lesioned_sim)
importlib.reload(init_params_tensor)
importlib.reload(compute_prob_tensor)
importlib.reload(main_sim_tensor)
importlib.reload(proxy_sim)

test_stimuli = init_model_tensor.granch_stimuli(3, "BBBBBD")
test_stimuli.add_toy_example(0.3, 0.7)

tensor_model =  init_model_tensor.granch_model(PRIOR_INFO['max_observation'], tensor_stimuli[0])

params = init_params_tensor.granch_params(
                grid_mu =  BATCH_GRID_INFO["grid_mus"][0].to(device),
                grid_sigma = BATCH_GRID_INFO["grid_sigmas"][0].to(device),
                grid_y = BATCH_GRID_INFO["grid_ys"][0].to(device),
                grid_epsilon = BATCH_GRID_INFO["grid_epsilons"][0].to(device),
                hypothetical_obs_grid_n = PRIOR_INFO["hypothetical_obs_grid_n"], 
                mu_prior = PRIOR_INFO["mu_prior"],
                V_prior = PRIOR_INFO["V_prior"], 
                alpha_prior = PRIOR_INFO["alpha_prior"], 
                beta_prior = PRIOR_INFO["beta_prior"],
                epsilon  = PRIOR_INFO["epsilon"], 
                mu_epsilon = PRIOR_INFO["mu_epsilon"], 
                sd_epsilon = PRIOR_INFO["sd_epsilon"], 
                world_EIGs = PRIOR_INFO["world_EIGs"],
                max_observation = PRIOR_INFO["max_observation"], 
                forced_exposure_max= np.nan, 
                linking_hypothesis= "surprisal")
        
            # add the various different cached bits
params.add_meshed_grid()
params.add_lp_mu_sigma()
params.add_y_given_mu_sigma()
params.add_lp_epsilon()
params.add_priors()

#res = main_sim_tensor.granch_main_simulation(params, tensor_model, test_stimuli)
res = proxy_sim.granch_proxy_sim(params, tensor_model, test_stimuli)
res.behavior



normalized prior
tensor([[[[-43123080.0000,  -8419467.0000, -43321860.0000,  ...,
            -4889016.5000, -10906330.0000, -37112452.0000],
          [-43123080.0000,  -8419467.0000, -43321860.0000,  ...,
            -4889016.5000, -10906330.0000, -37112452.0000],
          [-43123088.0000,  -8419473.0000, -43321868.0000,  ...,
            -4889022.5000, -10906336.0000, -37112460.0000],
          ...,
          [-43123080.0000,  -8419468.0000, -43321860.0000,  ...,
            -4889016.5000, -10906331.0000, -37112452.0000],
          [-43123168.0000,  -8419554.0000, -43321948.0000,  ...,
            -4889103.5000, -10906417.0000, -37112540.0000],
          [-43123108.0000,  -8419495.0000, -43321888.0000,  ...,
            -4889043.5000, -10906358.0000, -37112480.0000]],

         [[-43123088.0000,  -8419472.0000, -43321868.0000,  ...,
            -4889021.0000, -10906335.0000, -37112460.0000],
          [-43123088.0000,  -8419473.0000, -43321868.0000,  ...,
            -4889022.0000,

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:292: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.197026675808013' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.all_observations.loc[self.current_t]  = Normal(current_stimulus, noise_epsilon).sample().tolist()
/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:292: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.13873030808863854' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.all_observations.loc[self.current_t]  = Normal(current_stimulus, noise_epsilon).sample().tolist()
/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:292: FutureWarn

posterior
tensor([[[[9.9999e-321, 9.9999e-321, 9.9999e-321,  ..., 9.9999e-321,
           9.9999e-321, 9.9999e-321],
          [9.9999e-321, 9.9999e-321, 9.9999e-321,  ..., 9.9999e-321,
           9.9999e-321, 9.9999e-321],
          [9.9999e-321, 9.9999e-321, 9.9999e-321,  ..., 9.9999e-321,
           9.9999e-321, 9.9999e-321],
          ...,
          [9.9999e-321, 9.9999e-321, 9.9999e-321,  ..., 9.9999e-321,
           9.9999e-321, 9.9999e-321],
          [9.9999e-321, 9.9999e-321, 9.9999e-321,  ..., 9.9999e-321,
           9.9999e-321, 9.9999e-321],
          [9.9999e-321, 9.9999e-321, 9.9999e-321,  ..., 9.9999e-321,
           9.9999e-321, 9.9999e-321]],

         [[9.9999e-321, 9.9999e-321, 9.9999e-321,  ..., 9.9999e-321,
           9.9999e-321, 9.9999e-321],
          [9.9999e-321, 9.9999e-321, 9.9999e-321,  ..., 9.9999e-321,
           9.9999e-321, 9.9999e-321],
          [9.9999e-321, 9.9999e-321, 9.9999e-321,  ..., 9.9999e-321,
           9.9999e-321, 9.9999e-321],
          

,stimulus_id,EIG,Look_away
0,0,0.0,True
1,1,0.0,True
2,2,0.0,True
3,3,0.0,True
4,4,0.0,True
5,5,0.0,True
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN


In [52]:
torch.sum(torch.tensor([1, 1, 1]))

tensor(3)

RuntimeError: torch.meshgrid: Expected 0D or 1D tensor in the tensor list but got:  3  5
 3  6
 4  5
 4  6
[ torch.LongTensor{4,2} ]